In [1]:
import os

In [2]:
pwd

'c:\\Users\\kgn\\OneDrive - PowerSchool\\PowerSchool\\Release Script\\Python\\Data Scientist\\TextSummarizer\\research'

In [3]:
os.chdir("../")

In [4]:
pwd


'c:\\Users\\kgn\\OneDrive - PowerSchool\\PowerSchool\\Release Script\\Python\\Data Scientist\\TextSummarizer'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file: Path
    unzip_dir:Path